## Módulo: Analytics Engineering
    
## Aula 5


### Programação da Aula 5:

> ### 1. **O que é o Data Build Tool - DBT**;
> ### 2. **DBT na prática**;
> ### 3. **Desenvolvimento do projeto final**.

#### Link para o formulário para informar os integrantes do grupo do projeto:
https://forms.gle/xgT27QH81dxSrZ1w7

#### Link para o formulário de Feedback da aula:
https://forms.gle/TTcxvxQTUYAfc6kn7

## O que é o Data Build Tool - DBT

### O DBT é uma ferramenta de construção de dados que nos permite definir, testar e documentar nossas transformações de dados. Com esta ferramenta, podemos definir modelos de dados, transformar e preparar dados, e também testar nossas transformações para garantir que nossos dados estejam corretos.

<div style="text-align: center;">

<br>

<img src="https://www.analytics8.com/wp-content/uploads/2022/08/dbt-ecosystem-overview-Analytics8-scaled.jpg"  width="80%" height="60%">


### Ambiente virtual Python - Comandos executados no terminal

+ python -m venv venv 
+ .\venv\Scripts\activate


### criar um banco de dados no Postgres

+ create database postgres demo_dbt


### DBT na prática

### Instalação da biblioteca do DBT com o plugin do Postgres

In [ ]:
pip install dbt-core dbt-postgres

In [ ]:
dbt --version

dbt init demo_dbt

#### Verificar se foram criados as pastas:

+ ##### analysis
+ ##### seeds
+ ##### macros
+ ##### models
+ ##### tests

#### Caso contrário criar

#### Criar o arquivo dbt_project.yml - caso não exista

##### Conteúdo:



In [ ]:
default:
  outputs:
    dev:
      dbname: demo_dbt
      host: localhost
      pass: ada
      port: 5432
      schema: public
      threads: 10
      type: postgres
      user: postgres
  target: dev

dbt_project.yml
name: 'demo_dbt'

config-version: 2
version: '0.1'

profile: 'default'

model-paths: ["models"]
seed-paths: ["seeds"]
test-paths: ["tests"]
analysis-paths: ["analysis"]
macro-paths: ["macros"]

target-path: "target"
clean-targets:
    - "target"
    - "dbt_modules"
    - "logs"

require-dbt-version: [">=1.0.0", "<2.0.0"]

models:
  demo_dbt:
      materialized: table
      staging:
        materialized: view


#### Criar o arquivo C:\Users\ljsmo\\.dbt\profiles.yml - caso não exista

##### Conteúdo:

##### trocar ljsmo para o seu usuário

In [ ]:
default:
  outputs:
    dev:
      dbname: demo_dbt
      host: localhost
      pass: ada
      port: 5432
      schema: public
      threads: 10
      type: postgres
      user: postgres
  target: dev


In [3]:
dbt debug



#### Copiar os arquivos para a pasta seed

+ ##### raw_customers.csv
+ ##### raw_orders.csv
+ ##### raw_payments.csv

In [4]:
dbt seed



##### Verificar a criação no postgres

##### Criar a pasta postgres_sample_data dentro da pasta models 

##### criar arquivo schema.yml - Conteúdo:

In [ ]:
version: 2

sources:
  - name: demo_dbt
    description: 'Exemplo de dados fornecido pelo Postgres'
    database: demo_dbt
    schema: public 
    tables:
      - name: raw_orders
        description: 'Detalhes sobre as vendas na Loja.



##### Criar o arquivo postgres_sample_data__store_sales.sql dentro da pasta models dentro da pasta criada anteriormente - Conteúdo do arquivo:

In [ ]:
with source_store_sales as (
    select * from {{ source('demo_dbt', 'raw_orders')}}
),


final as (
    select * from source_store_sales
)


select * from final



In [ ]:
dbt run

##### Na pasta target → compiled → demo_dbt → models → postgres_sample_data existe um arquivo sql compilado pelo dbt


#### Vamos alterar o schema do postgres para ficar de acordo com o assunto de qualidade de dados - Vamos chamar de staging

##### No arquivo dbt_project.yml alterar o final para:



In [ ]:
models:
  demo_dbt:
        +materialized: view
        +schema: staging #PUBLIC_staging

In [ ]:
dbt run

Criando macro para gerar schema
+ Criar arquivo: generate_schema_name.sql dentro da pasta macros, conteúdo:



In [ ]:
{% macro generate_schema_name(custom_schema_name, node) -%}


    {%- set default_schema = target.schema -%}
    {%- if custom_schema_name is none -%}


        {{ default_schema }}


    {%- else -%}


        {{ custom_schema_name | trim }}


    {%- endif -%}


{%- endmacro %}

#### Criar a pasta staging dentro de models, colocar dentro da pasta os arquivos:

+ postgres_sample_data__store_sales.sql
+ schema.yml (pasta postgres_sample_data)

#### em dbt_project.yml mudar para

In [ ]:
models:
  demo_dbt:
    staging:
        +materialized: view
        +schema: staging #PUBLIC_staging


##### staging: é coincidente com o nome da pasta em modesls

In [ ]:
dbt run

##### Alterar o schema.yml no final para:

In [ ]:
    tables:
      - name: raw_orders
        description: 'Detalhes sobre as vendas na Loja.'
      - name: raw_payments
        description: 'Detalhes sobre os pagamentos das vendas na Loja.'

##### dentro do models/staging

+ ##### Criar um arquivo example_payment_method.sql

In [ ]:
{{ config(materialized='table' )}}


with source_payment_method as (
    select distinct payment_method from {{ source('demo_dbt', 'raw_payments')}}
),


final as (
    select distinct payment_method from source_payment_method
)


select distinct payment_method from final


In [ ]:
dbt run

#### Criar em models → staging → exemple_vendas_join.sql

In [ ]:
{{ config(materialized='table' )}}


with
ordem as (
    select id, order_date, status from {{ source('demo_dbt', 'raw_orders')}} where status = 'completed'
),


payment as (
    select order_id, payment_method, amount from {{ source('demo_dbt', 'raw_payments')}}
),


total as (
    select
        ordem.order_date,
        payment.payment_method,
        sum(payment.amount) as total_data
    from ordem
    left join payment on ordem.id = payment.order_id
    group by ordem.order_date, payment.payment_method
    order by ordem.order_date


),


final as (
    select * from total
)


select * from final

In [ ]:
dbt run

#### Testes

+ ##### Em tests criar um arquivo: payments_zero.sql

In [ ]:
SELECT id
from {{ source('demo_dbt', 'raw_payments')}}
where amount <= 0

In [ ]:
dbt test

##### Teste Generic
+ ##### em schema.yml acrescentar

In [ ]:
 - name: raw_customers
        description: 'Detalhes dos clientes da Loja.'


+ ##### Criar uma pasta generic em tests
+ ##### criar um arquivo:string_not_empty.sql

In [ ]:
string_not_empty.sql
{% test string_not_empty(model, column_name) %}
    select {{ column_name }}
    from {{ model }}
    where TRIM({{ column_name }}) = ''
{% endtest %}


+ ##### criar em models arquivo yml: oms_config.yml

In [ ]:
models:
  - name: raw_customers
    columns:
      - name: last_name
        tests:
          - string_not_empty


  - name: raw_orders
    columns:
      - name: id
        tests:
          - unique
          - not_null
  - name: id
    tests:
      - relationships:
          to: ref('raw_orders')
          field: user_id
  - name: raw_payments
    columns:
      - name: payment_method
        tests:
          - accepted_values:
             values: ['bank_transfer', 'coupon', 'credit_card']


In [ ]:
dbt test

#### Macros

+ ##### Criar um arquivo dentro de macros: Total_vendas.sql

In [ ]:
{% macro generate_total_vendas_por_metodo_pagamento(table_name) %}
select
    payment_method,
    sum(amount) as total
from {{ source  ('demo_dbt', table_name)}}
group by payment_method
{% endmacro %}


+ ##### dentro de models staging criar arquivo: tot_vendas.sql

In [ ]:
{{ generate_total_vendas_por_metodo_pagamento('raw_payments') }}

In [ ]:
dbt run

#### Pacotes -  https://hub.getdbt.com/dbt-labs/dbt_utils/latest/

+ ##### criar na raiz um arquivo packages.yml com o conteúdo do pacote a instalar:

In [ ]:
packages:
  - package: dbt-labs/dbt_utils
    version: 1.1.1

In [ ]:
dbt deps

#### escolhendo o deduplicate: https://github.com/dbt-labs/dbt-utils/tree/1.1.1/#deduplicate-source

+ ##### Criar um arquivo em model staging: dbt_pac.sql

In [ ]:
{{ dbt_utils.deduplicate(
    relation=source('demo_dbt', 'raw_orders'),
    partition_by='user_id',
    order_by="order_date desc",
   )
}}


In [ ]:
dbt run

#### Documentação

In [ ]:
dbt docs generate
dbt docs serve